V2: Fixed corner case in image-wise metric and also added global f2 score implementation.  
    Thanks for pointing out @lobrien
    
V3: Fixed a stupid bug in fbeta score calculation, thanks @philippsinger!

V7: There was still bug in the function, fixed by @yoichi7yamakawa. Thanks for pointing out!

In [ ]:
import glob
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 30, 30
np.set_printoptions(precision=3, suppress=True)

In [ ]:
df = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv')
df.head()

## visualization snnipet

In [ ]:
from ast import literal_eval


def load_image(video_id, video_frame, image_dir):
    img_path = f'{image_dir}/video_{video_id}/{video_frame}.jpg'
    assert os.path.exists(img_path), f'{img_path} does not exist.'
    img = cv2.imread(img_path)
    return img


def decode_annotations(annotaitons_str):
    """decode annotations in string to list of dict"""
    return literal_eval(annotaitons_str)

def load_image_with_annotations(video_id, video_frame, image_dir, annotaitons_str):
    img = load_image(video_id, video_frame, image_dir)
    annotations = decode_annotations(annotaitons_str)
    if len(annotations) > 0:
        for ann in annotations:
            cv2.rectangle(img, (ann['x'], ann['y']),
                (ann['x'] + ann['width'], ann['y'] + ann['height']),
                (255, 0, 0), thickness=2,)
    return img

def draw_predictions(img, pred_bboxes):
    img = img.copy()
    if len(pred_bboxes) > 0:
        for bbox in pred_bboxes:
            conf = bbox[0]
            x, y, w, h = bbox[1:].round().astype(int)
            cv2.rectangle(img, (x, y),
                (x+w, y+h),
                (0, 255, 255), thickness=2,)
            cv2.putText(
                img,
                f"{conf:.2}",
                (x, max(0, y-5)),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 0, 255),
                thickness=1,
            )
    return img

# test
# index = 0
index = 19668
# index = 16
row = df.iloc[index]
video_id = row.video_id
video_frame = row.video_frame
annotations_str = row.annotations
image_dir = '../input/tensorflow-great-barrier-reef/train_images'
img = load_image_with_annotations(video_id, video_frame, image_dir, annotations_str)
plt.imshow(img[:, :, ::-1])

## generate ground truth and dummy prediction

In [ ]:
def generate_gt_and_pred(annotations_str):
    annotations = decode_annotations(annotations_str)

    gt_bboxes = []
    pred_bboxes = []

    for ann in annotations:
        gt_bboxes.append(np.array([ann['x'], ann['y'], ann['width'], ann['height']]))

        # pseudo pred bbox
        conf = np.random.uniform()
#         noise = (np.random.randn(4)*5).round()
        pred_bbox = np.array([conf, ann['x'], ann['y'], ann['width']*1.25, ann['height']*1.25])
#         pred_bbox[1:] = pred_bbox[1:] + noise
        pred_bboxes.append(pred_bbox)

    gt_bboxes = np.array(gt_bboxes)
    pred_bboxes = np.array(pred_bboxes)
    return gt_bboxes, pred_bboxes

annotations_str = df.iloc[index]['annotations']
gt_bboxes, pred_bboxes = generate_gt_and_pred(annotations_str)

if len(pred_bboxes) > 0:
    pred_bboxes = pred_bboxes[pred_bboxes[:,0].argsort()[::-1]] # sort by conf
    pred_img = draw_predictions(img, pred_bboxes)
    plt.imshow(pred_img[:, :, ::-1])

## image wise metric implementation

f2 score is the case of beta=2 in fbeta score.  
https://en.wikipedia.org/wiki/F-score

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/6fbeb471033fdd63a2c2ca7830afc7abdf8b8134)

In [ ]:
def calc_iou(bboxes1, bboxes2, bbox_mode='xywh'):
    assert len(bboxes1.shape) == 2 and bboxes1.shape[1] == 4
    assert len(bboxes2.shape) == 2 and bboxes2.shape[1] == 4
    
    bboxes1 = bboxes1.copy()
    bboxes2 = bboxes2.copy()
    
    if bbox_mode == 'xywh':
        bboxes1[:, 2:] += bboxes1[:, :2]
        bboxes2[:, 2:] += bboxes2[:, :2]

    x11, y11, x12, y12 = np.split(bboxes1, 4, axis=1)
    x21, y21, x22, y22 = np.split(bboxes2, 4, axis=1)
    xA = np.maximum(x11, np.transpose(x21))
    yA = np.maximum(y11, np.transpose(y21))
    xB = np.minimum(x12, np.transpose(x22))
    yB = np.minimum(y12, np.transpose(y22))
    interArea = np.maximum((xB - xA + 1), 0) * np.maximum((yB - yA + 1), 0)
    boxAArea = (x12 - x11 + 1) * (y12 - y11 + 1)
    boxBArea = (x22 - x21 + 1) * (y22 - y21 + 1)
    iou = interArea / (boxAArea + np.transpose(boxBArea) - interArea)
    return iou

def f_beta(tp, fp, fn, beta=2):
    return (1+beta**2)*tp / ((1+beta**2)*tp + beta**2*fn+fp)

def imagewise_f2_score_at_iou_th(gt_bboxes, pred_bboxes, iou_th, verbose=False):
    gt_bboxes = gt_bboxes.copy()
    pred_bboxes = pred_bboxes.copy()

    tp = 0
    fp = 0
    for k, pred_bbox in enumerate(pred_bboxes): # fixed in ver.7
        ious = calc_iou(gt_bboxes, pred_bbox[None, 1:])
        max_iou = ious.max()
        if max_iou > iou_th:
            tp += 1
            gt_bboxes = np.delete(gt_bboxes, ious.argmax(), axis=0)
        else:
            fp += 1
        if len(gt_bboxes) == 0:
            fp += len(pred_bboxes) - (k + 1) # fix in ver.7
            break

    fn = len(gt_bboxes)
    score = f_beta(tp, fp, fn, beta=2)
    if verbose:
        print(f'iou_th:{iou_th.round(2):<4} tp:{tp:<2}, fp:{fp:<2}, fn:{fn:<2} f2:{score:.3}')
    return score



def imagewise_f2_score(gt_bboxes, pred_bboxes, verbose=False):
    """
    gt_bboxes: (N, 4) np.array in xywh format
    pred_bboxes: (N, 5) np.array in conf+xywh format
    """
    # v2: add corner case hundling.
    if len(gt_bboxes) == 0 and len(pred_bboxes) == 0:
        return 1.0
    elif len(gt_bboxes) == 0 or len(pred_bboxes) == 0:
        return 0.0
    
    pred_bboxes = pred_bboxes[pred_bboxes[:,0].argsort()[::-1]] # sort by conf
    
    scores = []
    for iou_th in np.arange(0.3, 0.85, 0.05):
        scores.append(imagewise_f2_score_at_iou_th(gt_bboxes, pred_bboxes, iou_th, verbose))
    return np.mean(scores)

In [ ]:
imagewise_f2_score(gt_bboxes, pred_bboxes, verbose=True)

# V2: Global f2 metric implementation
In version 1 I've implemented image-wise metric, but competition metric shoud gather tp/fp/fn over all images and then calculate f2 score.  
It means above metric is somehow wrong, but I'll keep it here as it might be useful for someone want to calculate image-wise metric.

Anyway, let's implement correct metric:)

In [ ]:
def calc_iou(bboxes1, bboxes2, bbox_mode='xywh'):
    assert len(bboxes1.shape) == 2 and bboxes1.shape[1] == 4
    assert len(bboxes2.shape) == 2 and bboxes2.shape[1] == 4
    
    bboxes1 = bboxes1.copy()
    bboxes2 = bboxes2.copy()
    
    if bbox_mode == 'xywh':
        bboxes1[:, 2:] += bboxes1[:, :2]
        bboxes2[:, 2:] += bboxes2[:, :2]

    x11, y11, x12, y12 = np.split(bboxes1, 4, axis=1)
    x21, y21, x22, y22 = np.split(bboxes2, 4, axis=1)
    xA = np.maximum(x11, np.transpose(x21))
    yA = np.maximum(y11, np.transpose(y21))
    xB = np.minimum(x12, np.transpose(x22))
    yB = np.minimum(y12, np.transpose(y22))
    interArea = np.maximum((xB - xA + 1), 0) * np.maximum((yB - yA + 1), 0)
    boxAArea = (x12 - x11 + 1) * (y12 - y11 + 1)
    boxBArea = (x22 - x21 + 1) * (y22 - y21 + 1)
    iou = interArea / (boxAArea + np.transpose(boxBArea) - interArea)
    return iou

def f_beta(tp, fp, fn, beta=2):
    return (1+beta**2)*tp / ((1+beta**2)*tp + beta**2*fn+fp)

def calc_is_correct_at_iou_th(gt_bboxes, pred_bboxes, iou_th, verbose=False):
    gt_bboxes = gt_bboxes.copy()
    pred_bboxes = pred_bboxes.copy()
    
    tp = 0
    fp = 0
    for k, pred_bbox in enumerate(pred_bboxes): # fixed in ver.7
        ious = calc_iou(gt_bboxes, pred_bbox[None, 1:])
        max_iou = ious.max()
        if max_iou > iou_th:
            tp += 1
            gt_bboxes = np.delete(gt_bboxes, ious.argmax(), axis=0)
        else:
            fp += 1
        if len(gt_bboxes) == 0:
            fp += len(pred_bboxes) - (k + 1) # fix in ver.7
            break

    fn = len(gt_bboxes)
    return tp, fp, fn

def calc_is_correct(gt_bboxes, pred_bboxes):
    """
    gt_bboxes: (N, 4) np.array in xywh format
    pred_bboxes: (N, 5) np.array in conf+xywh format
    """
    if len(gt_bboxes) == 0 and len(pred_bboxes) == 0:
        tps, fps, fns = 0, 0, 0
        return tps, fps, fns
    
    elif len(gt_bboxes) == 0:
        tps, fps, fns = 0, len(pred_bboxes), 0
        return tps, fps, fns
    
    elif len(pred_bboxes) == 0:
        tps, fps, fns = 0, 0, len(gt_bboxes)
        return tps, fps, fns
    
    pred_bboxes = pred_bboxes[pred_bboxes[:,0].argsort()[::-1]] # sort by conf
    
    tps, fps, fns = 0, 0, 0
    for iou_th in np.arange(0.3, 0.85, 0.05):
        tp, fp, fn = calc_is_correct_at_iou_th(gt_bboxes, pred_bboxes, iou_th)
        tps += tp
        fps += fp
        fns += fn
    return tps, fps, fns

def calc_f2_score(gt_bboxes_list, pred_bboxes_list, verbose=False):
    """
    gt_bboxes_list: list of (N, 4) np.array in xywh format
    pred_bboxes_list: list of (N, 5) np.array in conf+xywh format
    """
    tps, fps, fns = 0, 0, 0
    for gt_bboxes, pred_bboxes in zip(gt_bboxes_list, pred_bboxes_list):
        tp, fp, fn = calc_is_correct(gt_bboxes, pred_bboxes)
        tps += tp
        fps += fp
        fns += fn
        if verbose:
            num_gt = len(gt_bboxes)
            num_pred = len(pred_bboxes)
            print(f'num_gt:{num_gt:<3} num_pred:{num_pred:<3} tp:{tp:<3} fp:{fp:<3} fn:{fn:<3}')
    return f_beta(tps, fps, fns, beta=2)

### Generate gt bboxes list and dummy pred bboxes list

In [ ]:
gt_df = df.iloc[15:20]
display(gt_df)

gt_bboxes_list = []
pred_bboxes_list = []
for ann_str in gt_df['annotations']:
    gt_bboxes, pred_bboxes = generate_gt_and_pred(ann_str)
    gt_bboxes_list.append(gt_bboxes)
    pred_bboxes_list.append(pred_bboxes)

In [ ]:
calc_f2_score(gt_bboxes_list, pred_bboxes_list, verbose=True)